In [1]:
# Cell 1: Imports
import os
import glob
import shutil

# Standard LangChain Imports
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

# Retrievers
from langchain_community.retrievers import BM25Retriever

# Try importing EnsembleRetriever from s
from langchain_classic.retrievers import EnsembleRetriever

print("✅ Imports successful!")

✅ Imports successful!


In [2]:
# Cell 2: Setup Embeddings & Splitter
print("📥 Loading Hugging Face Embedding Model...")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Define headers to split on
headers_to_split_on = [
    ("#", "DocName"),
    ("##", "Section"),
    ("###", "SubSection"),
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
print("✅ Models & Splitter Ready.")

📥 Loading Hugging Face Embedding Model...
✅ Models & Splitter Ready.


In [3]:
# Cell 3: Process Files (Recursive)
all_splits = []
data_folder = "data" 

# FIX: Use recursive search to find files in subfolders (like data/departments/)
# "root_dir/**/*.md" will find matching files in any subdirectory
md_files = glob.glob(os.path.join(data_folder, "**/*.md"), recursive=True)

print(f"📂 Found {len(md_files)} Markdown files in '{data_folder}' and its subfolders.")

for file_path in md_files:
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            file_content = f.read()
        
        # Split text based on headers
        splits = markdown_splitter.split_text(file_content)
        
        # Metadata Injection
        for split in splits:
            # Add source filename
            split.metadata["source"] = os.path.basename(file_path)
            
            # Create a context string from headers
            header_context = ""
            if "DocName" in split.metadata:
                header_context += f"Document: {split.metadata['DocName']}\n"
            if "Section" in split.metadata:
                header_context += f"Section: {split.metadata['Section']}\n"
            if "SubSection" in split.metadata:
                header_context += f"Topic: {split.metadata['SubSection']}\n"
            
            # Prepend to content
            split.page_content = header_context + "\n" + split.page_content
            
        all_splits.extend(splits)
        print(f"   ✅ Processed {os.path.basename(file_path)} -> {len(splits)} chunks.")
        
    except Exception as e:
        print(f"   ❌ Error reading {file_path}: {e}")

print(f"Total Chunks to Index: {len(all_splits)}")

📂 Found 23 Markdown files in 'data' and its subfolders.
   ✅ Processed about_college_accreditations.md -> 6 chunks.
   ✅ Processed academic_regulations.md -> 8 chunks.
   ✅ Processed admissions_process.md -> 12 chunks.
   ✅ Processed campus_facilities.md -> 7 chunks.
   ✅ Processed departments_overview.md -> 9 chunks.
   ✅ Processed eligibility_criteria.md -> 11 chunks.
   ✅ Processed fee_structure.md -> 8 chunks.
   ✅ Processed governance_and_contact.md -> 18 chunks.
   ✅ Processed hostel_transport.md -> 7 chunks.
   ✅ Processed placements_statistics.md -> 7 chunks.
   ✅ Processed reserach_innovation.md -> 4 chunks.
   ✅ Processed student_life.md -> 6 chunks.
   ✅ Processed ai_ds_department.md -> 11 chunks.
   ✅ Processed ai_ml_department.md -> 11 chunks.
   ✅ Processed civil_department.md -> 12 chunks.
   ✅ Processed cse_department.md -> 13 chunks.
   ✅ Processed cyber_department.md -> 10 chunks.
   ✅ Processed ece_department.md -> 13 chunks.
   ✅ Processed eee_department.md -> 13 ch

In [4]:
# Cell 4: Create Vector Database
if os.path.exists("./chroma_db"):
    shutil.rmtree("./chroma_db")
    print("   🗑️  Cleared old database.")

print(f"⏳ Ingesting {len(all_splits)} chunks into ChromaDB...")
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
    persist_directory="./chroma_db"
)
print("🎉 Database Created Successfully!")

   🗑️  Cleared old database.
⏳ Ingesting 230 chunks into ChromaDB...
🎉 Database Created Successfully!


In [5]:
# Cell 5: Hybrid Retriever Setup

# 1. Vector Retriever (Semantic)
vector_retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# 2. Keyword Retriever (BM25)
# Note: We pass 'all_splits' directly to ensure 100% indexing
bm25_retriever = BM25Retriever.from_documents(all_splits)
bm25_retriever.k = 3

# 3. Ensemble (Hybrid)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.5, 0.5]
)

print("✅ Hybrid Retrieval System Online")

✅ Hybrid Retrieval System Online


In [10]:
# Cell 6: Test Query
query = "My EAPCET rank is 45,000 and I am from the BC-D category. Can I get a seat in CSE or ECE?"
print(f"\n🔎 SEARCHING FOR: '{query}'")
print("="*60)

results = ensemble_retriever.invoke(query)

for i, doc in enumerate(results):
    print(f"🔹 Result #{i+1}")
    print(f"   Source: {doc.metadata.get('source', 'Unknown')}")
    # Print first 200 characters to verify context injection worked
    print(f"   📄 Content: {doc.page_content}...") 
    print("-" * 50)


🔎 SEARCHING FOR: 'My EAPCET rank is 45,000 and I am from the BC-D category. Can I get a seat in CSE or ECE?'
🔹 Result #1
   Source: governance_and_contact.md
   📄 Content: Document: Governance, Leadership & Contact Info - Ramachandra College of Engineering (RCEE)
Section: 2. Administration (Academic Leadership)
Topic: Director-Principal

* **Name:** **Dr. M. Muralidhara Rao**
* **Qualification:** B.E., M.E., Ph.D. (IIT-Madras).
* **Memberships:** F.I.E. (India), M.I.E.T. (UK), ISTE Life Member.
* **Search Tags:** **Princiapl, Principal, princpal, MuraliDhar, MuarliDhar, Director, Directr**
* **Role:** Head of Academics & Administration Principal.
* **Contact:** principal@rcee.ac.in
* **Profile & Experience:**
* **Experience:** **40 years** in Academics and Administration. Served as Principal for two decades.
* **Expertise:** Complete knowledge of accreditations (NAAC, NBA) and research funding (DST, AICTE, STEM).
* **Research:** Active researcher; has guided four scholars.
* **Technic